In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import gridspec
from matplotlib import patches

import os
import math
import random

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.metrics import roc_curve, roc_auc_score, auc

import scipy.stats as st
from scipy.stats import poisson
from scipy.interpolate import interp1d
from scipy.stats import norm
from scipy.stats import multivariate_normal

In [2]:
from sklearn.neighbors import KernelDensity # paquete necesario
from sklearn.model_selection import GridSearchCV

import scipy.integrate as integrate

### DATA

In [ ]:
# you need
pred_XG_SM # background test-dataset (output of the ML)
pred_XG_NP # signal test-dataset (output of the ML)

B_expected # number of background events expected in your experiment
S_expected # number of signal events expected in your experiment

### SOME PARAMETERS

In [3]:
ntrials = 5000 # number of iterations, to search the optimal binning

MIN_EVS = 5 # minimum number of events per bin

n_ensembles = 5000 # number of pseudo experiments to compute Z

### Determine number of bins with 'bin_edges'

In [ ]:
print(' NUMBER OF BINS')

divs_B = int(len(pred_XG_SM)/B_expected)

B_1 = []
B_2 = []
B_3 = []

for it in range(divs_B):
    datB_grid_SM = pred_XG_SM[(B_expected*it):(B_expected*(it+1))]
    datB_grid_NP = pred_XG_NP[:B_expected]

    B_hist1 = np.histogram_bin_edges(datB_grid_SM, bins = 'fd')
    B_hist2 = np.histogram_bin_edges(datB_grid_SM, bins = 'doane')
    B_hist3 = np.histogram_bin_edges(datB_grid_SM, bins = 'sturges')
    
    B_1.append(len(B_hist1))
    B_2.append(len(B_hist2))
    B_3.append(len(B_hist3))



B_1_mean = int(np.mean(B_1))
B_2_mean = int(np.mean(B_2))
B_3_mean = int(np.mean(B_3))


B_hist1 = plt.hist(datB_grid_SM, bins = B_1_mean, histtype = 'step', label = 'fd')
B_hist2 = plt.hist(datB_grid_SM, bins = B_2_mean, histtype = 'step', label = 'doane')
B_hist3 = plt.hist(datB_grid_SM, bins = B_3_mean, histtype = 'step', label = 'sturges')
plt.legend()
plt.show()


print('fd', B_1_mean)
print('doane', B_2_mean)
print('sturges', B_3_mean)
print(' ')


# SAVE THE OPTIMAL BIN NUMBER
B_bins_mean = [B_1_mean, B_2_mean, B_3_mean]

### BIN LIKELIHOOD with the 'optimal' number of bins (equal size bins)

#### without statistical error

In [ ]:
# SAVED IN:
# Z_bins_XG_CV # asking for a min number of events per bin
# Z_bins_XG_CV_zeros # replacing the zeros

In [ ]:
###############################
# EQ SIZE CROSS-VAL FOR Nbins #
###############################

print('\n BINNING: eq size bins, with the optimal number of bins: ')


print('minimum number of events per bin: ', MIN_EVS)

print('')



Z_bins_XG_CV = []
Z_bins_XG_CV_zeros = []




# Les't find the number of possible ensembles
N_ensembles_back = len(pred_XG_SM) / B_expected
N_ensembles_sig = len(pred_XG_NP) / S_expected



for j_it in range(len(B_bins_mean)):
    
    # bin the parameter space of all background events
    hist_back, binedges_back = np.histogramdd([pred_XG_SM], bins=(B_bins_mean[j_it]), range = [[min(pred_XG_SM),max(pred_XG_SM)]])
    bin_edges = binedges_back[0]
    
    if min(hist_back) >= MIN_EVS*N_ensembles_back:
        print('ok j_it=', j_it)
        
        # now divide by the number of possible ensembles
        back_prom = hist_back/N_ensembles_back

        # same for signal
        hist_sig, binedges_sig = np.histogramdd([pred_XG_NP], bins=[bin_edges])
        sig_prom = hist_sig/N_ensembles_sig

        # then the signif Z^binned-Asimov:
        Z_bins_XG_CV_aux = ( 2* sum( ( back_prom * np.log( back_prom / (sig_prom+back_prom) ) ) + sig_prom ) )**0.5

    else:
        print('NO ok j_it=', j_it)
        Z_bins_XG_CV_aux = 0
        
    Z_bins_XG_CV.append(Z_bins_XG_CV_aux)
    
    
    # REPLACING the zeros
    hist_back_noceros = []
    for i in range(len(hist_back)):
        if hist_back[i]!=0:
            hist_back_noceros.append(hist_back[i])

    min_back = min(hist_back_noceros)

    # replace the zeros
    for i in range(len(hist_back)):
        if hist_back[i]==0:
            hist_back[i] = min_back

    # now divide by the number of possible ensembles
    back_prom = hist_back/N_ensembles_back

    # same for signal
    hist_sig, binedges_sig = np.histogramdd([pred_XG_NP], bins=[bin_edges])
    sig_prom = hist_sig/N_ensembles_sig
    
    Z_bins_XG_CV_zeros.append( ( 2* sum( ( back_prom * np.log( back_prom / (sig_prom+back_prom) ) ) + sig_prom ) )**0.5 )



print(' ')

#### with statistical error

In [ ]:
# SAVED IN:
# Z_bins_XG_CV_stat # asking for a min number of events per bin
# Z_bins_XG_CV_stat_plus # central value +1sigma
# Z_bins_XG_CV_stat_min # central value -1sigma

# Z_bins_XG_CV_stat_zeros # replacing the zeros
# Z_bins_XG_CV_stat_zeros_plus # central value +1sigma
# Z_bins_XG_CV_stat_zeros_min # central value -1sigma

In [ ]:
###############################
# EQ SIZE CROSS-VAL FOR Nbins #
###############################

print('\n BINNING: eq size bins, with the optimal number of bins: ')


print('minimum number of events per bin: ', MIN_EVS)

print('')



Z_bins_XG_CV_stat = []
Z_bins_XG_CV_stat_zeros = []

Z_bins_XG_CV_stat_std = []
Z_bins_XG_CV_stat_zeros_std = []



print('B_expected: ', B_expected)
print('S_expected: ', S_expected)


# to construct ensembles B and S events are taken from Poisson distributions
mu = S_expected + B_expected


# Letś find the number of events per ensemble such that we get at least one ensemble populated if events are taken from a Poisson distribution

# around the mean its populated so let's try (proposed range to be checked)
list_events_per_ensembles = [i for i in range(int(mu*0.9),int(mu*1.1))]
to_check = len(list_events_per_ensembles)

# I want at least one ensemble populated
list_nums_ensembles = [ int( poisson.pmf(list_events_per_ensembles[i],mu)*n_ensembles ) for i in range(len(list_events_per_ensembles)) ]



# Remove from the list the elements without at least 1 ensemble possible
for i in range(len(list_events_per_ensembles)):
    if list_nums_ensembles[i] > 1:
        list_events_per_ensembles = list_events_per_ensembles[i:]
        list_nums_ensembles = list_nums_ensembles[i:]
        break


for i in range(len(list_events_per_ensembles)):
    if list_nums_ensembles[i] < 1:
        list_events_per_ensembles = list_events_per_ensembles[:i]
        list_nums_ensembles = list_nums_ensembles[:i]
        break

print('\n If ', to_check, ' = ', len(list_events_per_ensembles), '   then the proposed range has to be extended')

print('n_ensembles (actual): ', sum(list_nums_ensembles))



# lists of S and B events per ensemble, w.r.t the total of number of events per ensemble found above:

p_berno = S_expected/(S_expected+B_expected)

list_S_per_ensembles = []
list_B_per_ensembles = []

for jj in range(len(list_events_per_ensembles)):
    list_S_per_ensembles.append( int(p_berno * list_events_per_ensembles[jj]) )
    list_B_per_ensembles.append( list_events_per_ensembles[jj] - int(p_berno * list_events_per_ensembles[jj]) )

######
# NOW I HAVE 4 LISTS:
# list_events_per_ensembles     list with the number of events per ensemble (its a range)
# list_nums_ensembles           list with the number of ensembles, w.r.t the 1st list
# list_S_per_ensembles          list with the number of signal events in each ensembles, w.r.t the 1st list
# list_B_per_ensembles          list with the number of background events in each ensembles, w.r.t the 1st list
######
    

Z_bins_XG_CV_stat_aux = []
Z_bins_XG_CV_stat_aux_zeros = []

for j_it in range(len(B_bins_mean)):
    
    for bb in range(len(list_nums_ensembles)):

        for kk in range(list_nums_ensembles[bb]):
            
            ran_ind_B = np.random.choice(indices_B, list_B_per_ensembles[bb])
            ran_ind_S = np.random.choice(indices_S, list_S_per_ensembles[bb])
            
            # estimate the variance in each bin as ~ (upB - downB)/2 
            
            pred_XG_SM_shuf = []
            
            pred_XG_NP_shuf = []
            
            for ill in ran_ind_B:
                pred_XG_SM_shuf.append(pred_XG_SM[ill])
                
            for ill in ran_ind_S:
                pred_XG_NP_shuf.append(pred_XG_NP[ill])
                
            

            # Let's find out the expected number of B and S events in each bin:       

            # bin the parameter space of all background events
            hist_back, binedges_back = np.histogramdd([pred_XG_SM_shuf], bins=(B_bins_mean[j_it]), range = [[min(pred_XG_SM_shuf),max(pred_XG_SM_shuf)]])
            bin_edges = binedges_back[0]
            
            back_prom = hist_back.T.ravel()
            
            
            if min(hist_back) >= MIN_EVS:
                print('ok j_it=', j_it)

                # same for signal
                hist_sig, binedges_sig = np.histogramdd([pred_XG_NP], bins=[bin_edges])
                sig_prom = hist_sig.T.ravel()

                # then the signif Z^binned-Asimov:
                Z_bins_XG_CV_stat_aux = ( 2* sum( ( back_prom * np.log( back_prom / (sig_prom+back_prom) ) ) + sig_prom ) )**0.5

            else:
                print('NO ok j_it=', j_it)

            



            # If a bins has no expected background events replace that zero for the minimum B_bin=/=0

            # find the minimum
            back_prom_noceros = []
            for i in range(len(back_prom)):
                if back_prom[i]!=0:
                    back_prom_noceros.append(back_prom[i])

            min_back = min(back_prom_noceros)

            # replace the zeros
            for i in range(len(back_prom)):
                if back_prom[i]==0:
                    back_prom[i] = min_back
                    
                    
            # same for signal
            hist_sig, binedges_sig = np.histogramdd([pred_XG_NP_shuf], bins=[bin_edges])
            sig_prom = hist_sig.T.ravel()
            
            Z_bins_XG_CV_stat_aux_zeros.append( ( 2* sum( ( back_prom * np.log( back_prom / (sig_prom+back_prom) ) ) + sig_prom ) )**0.5 )

                    
                    
    # Histogram of q_muhats
    
    print(' CHECK IF YOU HAVE ENOUGH PSEUDO EXPERIMENTS (cause you ask for a minimun number of events per bin): ')
    print('pseudo experiments tested: ', n_ensembles)
    print('pseudo experiments that have at least ', MIN_EVS, ' events per bin: ', len(Z_bins_XG_CV_stat_aux))
    print('\n you need at least... 1000? to describe the distibution')
    print(' if not, increase the number of pseudo experiments' )

    weights = np.ones_like(Z_bins_XG_CV_stat_aux)/float(len(Z_bins_XG_CV_stat_aux))
    nMIX, binsMIX, patchesMIX = plt.hist(Z_bins_XG_CV_stat_aux, 25, weights=weights, histtype='step', color='blue', linewidth=2)
    weights = np.ones_like(Z_bins_XG_CV_stat_aux_zeros)/float(len(Z_bins_XG_CV_stat_aux_zeros))
    nMIX, binsMIX, patchesMIX = plt.hist(Z_bins_XG_CV_stat_aux_zeros, 25, weights=weights, histtype='step', color='green', linewidth=2)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlabel("Z",fontsize=16)
    plt.ylabel("Fraction of ensembles",fontsize=16)
    plt.grid()
    #plt.legend(fontsize=14)
    plt.show()
    
    
    # Remove nan if any
    Z_bins_XG_CV_stat_aux = [x for x in Z_bins_XG_CV_stat_aux if x == x]
    for jk in range(len(Z_bins_XG_CV_stat_aux)):
        if Z_bins_XG_CV_stat_aux[jk] < 0:
            Z_bins_XG_CV_stat_aux[jk] = 0

    Z_bins_XG_CV_stat_median = np.median(Z_bins_XG_CV_stat_aux)   
    Z_bins_XG_CV_stat.append(Z_bins_XG_CV_stat_median)
    
    Z_bins_XG_CV_stat_stdX = np.std(Z_bins_XG_CV_stat_aux)   
    Z_bins_XG_CV_stat_std.append(Z_bins_XG_CV_stat_stdX)

    print('# bins: ', B_bins_mean[j_it])
    print('Z median: ', Z_bins_XG_CV_stat_median)
    print('Z std: ', Z_bins_XG_CV_stat_stdX)
    
    
    # Remove nan if any
    Z_bins_XG_CV_stat_aux_zeros = [x for x in Z_bins_XG_CV_stat_aux_zeros if x == x]
    for jk in range(len(Z_bins_XG_CV_stat_aux_zeros)):
        if Z_bins_XG_CV_stat_aux_zeros[jk] < 0:
            Z_bins_XG_CV_stat_aux_zeros[jk] = 0

    Z_bins_XG_CV_stat_zeros_median = np.median(Z_bins_XG_CV_stat_aux_zeros)   
    Z_bins_XG_CV_stat_zeros.append(Z_bins_XG_CV_stat_zeros_median)
    
    Z_bins_XG_CV_stat_zeros_stdX = np.std(Z_bins_XG_CV_stat_aux_zeros)   
    Z_bins_XG_CV_stat_zeros_std.append(Z_bins_XG_CV_stat_zeros_stdX)

    print('# bins: ', B_bins_mean[j_it])
    print('Z median: ', Z_bins_XG_CV_stat_zeros_median)
    print('Z std: ', Z_bins_XG_CV_stat_zeros_stdX)
    print('')

print(' ')


Z_bins_XG_CV_stat_plus = [i+j for i, j in zip(Z_bins_XG_CV_stat, Z_bins_XG_CV_stat_std)]
Z_bins_XG_CV_stat_min = [i-j for i, j in zip(Z_bins_XG_CV_stat, Z_bins_XG_CV_stat_std)]

Z_bins_XG_CV_stat_zeros_plus = [i+j for i, j in zip(Z_bins_XG_CV_stat_zeros, Z_bins_XG_CV_stat_zeros_std)]
Z_bins_XG_CV_stat_zeros_min = [i-j for i, j in zip(Z_bins_XG_CV_stat_zeros, Z_bins_XG_CV_stat_zeros_std)]

### BIN LIKELIHOOD with the 'optimal' number of bins (same number of background events per bin)

#### without statistical error

In [ ]:
# SAVED IN:
# Z_bins_XG_CV_eqBperbin # asking for a min number of events per bin

In [ ]:
##############################################
# EQ back events per bin CROSS-VAL FOR Nbins #
##############################################

print('\n BINNING: eq background events per bin, with the optimal number of bins: ')


print('minimum number of events per bin: ', MIN_EVS)

print('')



Z_bins_XG_CV_eqBperbin = []



# Les't find the number of possible ensembles
N_ensembles_back = len(pred_XG_SM) / B_expected
N_ensembles_sig = len(pred_XG_NP) / S_expected



for j_it in range(len(B_bins_mean)):
    
    # same number of B events per bin
    bin_edges_same = pd.qcut(pd.DataFrame(pred_XG_SM)[0] + jitter(pd.DataFrame(pred_XG_SM)[0]), q = B_bins_mean[j_it], precision=0, retbins = True)[1]
    bin_edges = bin_edges_same
    
    # bin the parameter space of all background events
    hist_back, binedges_back = np.histogramdd([pred_XG_NP], bins=[bin_edges])
    
    if min(hist_back) >= MIN_EVS*N_ensembles_back:
        print('ok j_it=', j_it)
        
        # now divide by the number of possible ensembles
        back_prom = hist_back/N_ensembles_back

        # same for signal
        hist_sig, binedges_sig = np.histogramdd([pred_XG_NP], bins=[bin_edges])
        sig_prom = hist_sig/N_ensembles_sig

        # then the signif Z^binned-Asimov:
        Z_bins_XG_CV_eqBperbin_aux = ( 2* sum( ( back_prom * np.log( back_prom / (sig_prom+back_prom) ) ) + sig_prom ) )**0.5

    else:
        print('NO ok j_it=', j_it)
        Z_bins_XG_CV_eqBperbin_aux = 0
        
    Z_bins_XG_CV_eqBperbin.append(Z_bins_XG_CV_eqBperbin_aux)
    
print(' ')

#### with statistical error

In [ ]:
# SAVED IN:
# Z_bins_XG_CV_eqBperbin_stat # asking for a min number of events per bin
# Z_bins_XG_CV_eqBperbin_stat_plus # central value +1sigma
# Z_bins_XG_CV_eqBperbin_stat_min # central value -1sigma

In [ ]:
##############################################
# EQ back events per bin CROSS-VAL FOR Nbins #
##############################################

print('\n BINNING: eq background events per bin, with the optimal number of bins: ')


print('minimum number of events per bin: ', MIN_EVS)

print('')


Z_bins_XG_CV_eqBperbin_stat = []
Z_bins_XG_CV_eqBperbin_stat_std = []



print('B_expected: ', B_expected)
print('S_expected: ', S_expected)


# to construct ensembles B and S events are taken from Poisson distributions
mu = S_expected + B_expected


# Letś find the number of events per ensemble such that we get at least one ensemble populated if events are taken from a Poisson distribution

# around the mean its populated so let's try (proposed range to be checked)
list_events_per_ensembles = [i for i in range(int(mu*0.9),int(mu*1.1))]
to_check = len(list_events_per_ensembles)

# I want at least one ensemble populated
list_nums_ensembles = [ int( poisson.pmf(list_events_per_ensembles[i],mu)*n_ensembles ) for i in range(len(list_events_per_ensembles)) ]



# Remove from the list the elements without at least 1 ensemble possible
for i in range(len(list_events_per_ensembles)):
    if list_nums_ensembles[i] > 1:
        list_events_per_ensembles = list_events_per_ensembles[i:]
        list_nums_ensembles = list_nums_ensembles[i:]
        break


for i in range(len(list_events_per_ensembles)):
    if list_nums_ensembles[i] < 1:
        list_events_per_ensembles = list_events_per_ensembles[:i]
        list_nums_ensembles = list_nums_ensembles[:i]
        break

print('\n If ', to_check, ' = ', len(list_events_per_ensembles), '   then the proposed range has to be extended')

print('n_ensembles (actual): ', sum(list_nums_ensembles))



# lists of S and B events per ensemble, w.r.t the total of number of events per ensemble found above:

p_berno = S_expected/(S_expected+B_expected)

list_S_per_ensembles = []
list_B_per_ensembles = []

for jj in range(len(list_events_per_ensembles)):
    list_S_per_ensembles.append( int(p_berno * list_events_per_ensembles[jj]) )
    list_B_per_ensembles.append( list_events_per_ensembles[jj] - int(p_berno * list_events_per_ensembles[jj]) )

######
# NOW I HAVE 4 LISTS:
# list_events_per_ensembles     list with the number of events per ensemble (its a range)
# list_nums_ensembles           list with the number of ensembles, w.r.t the 1st list
# list_S_per_ensembles          list with the number of signal events in each ensembles, w.r.t the 1st list
# list_B_per_ensembles          list with the number of background events in each ensembles, w.r.t the 1st list
######
    

Z_bins_XG_CV_stat_aux = []

for j_it in range(len(B_bins_mean)):
    
    for bb in range(len(list_nums_ensembles)):

        for kk in range(list_nums_ensembles[bb]):
            
            ran_ind_B = np.random.choice(indices_B, list_B_per_ensembles[bb])
            ran_ind_S = np.random.choice(indices_S, list_S_per_ensembles[bb])
            
            # estimate the variance in each bin as ~ (upB - downB)/2 
            
            pred_XG_SM_shuf = []
            
            pred_XG_NP_shuf = []
            
            for ill in ran_ind_B:
                pred_XG_SM_shuf.append(pred_XG_SM[ill])
                
            for ill in ran_ind_S:
                pred_XG_NP_shuf.append(pred_XG_NP[ill])
                
                
                
            # same number of B events per bin
            bin_edges_same = pd.qcut(pd.DataFrame(pred_XG_SM_shuf)[0] + jitter(pd.DataFrame(pred_XG_SM_shuf)[0]), q = B_bins_mean[j_it], precision=0, retbins = True)[1]
            bin_edges = bin_edges_same
    
            # bin the parameter space of all background events
            hist_back, binedges_back = np.histogramdd([pred_XG_NP], bins=[bin_edges])
            
            back_prom = hist_back.T.ravel()
            
            
            if min(hist_back) >= MIN_EVS:
                print('ok j_it=', j_it)

                # same for signal
                hist_sig, binedges_sig = np.histogramdd([pred_XG_NP], bins=[bin_edges])
                sig_prom = hist_sig.T.ravel()

                # then the signif Z^binned-Asimov:
                Z_bins_XG_CV_stat_aux = ( 2* sum( ( back_prom * np.log( back_prom / (sig_prom+back_prom) ) ) + sig_prom ) )**0.5

            else:
                print('NO ok j_it=', j_it)

            


                    
    # Histogram of q_muhats
    
    print(' CHECK IF YOU HAVE ENOUGH PSEUDO EXPERIMENTS (cause you ask for a minimun number of events per bin): ')
    print('pseudo experiments tested: ', n_ensembles)
    print('pseudo experiments that have at least ', MIN_EVS, ' events per bin: ', len(Z_bins_XG_CV_stat_aux))
    print('\n you need at least... 1000? to describe the distibution')
    print(' if not, increase the number of pseudo experiments' )

    weights = np.ones_like(Z_bins_XG_CV_stat_aux)/float(len(Z_bins_XG_CV_stat_aux))
    nMIX, binsMIX, patchesMIX = plt.hist(Z_bins_XG_CV_stat_aux, 25, weights=weights, histtype='step', color='blue', linewidth=2)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlabel("Z",fontsize=16)
    plt.ylabel("Fraction of ensembles",fontsize=16)
    plt.grid()
    #plt.legend(fontsize=14)
    plt.show()
    
    
    # Remove nan if any
    Z_bins_XG_CV_stat_aux = [x for x in Z_bins_XG_CV_stat_aux if x == x]
    for jk in range(len(Z_bins_XG_CV_stat_aux)):
        if Z_bins_XG_CV_stat_aux[jk] < 0:
            Z_bins_XG_CV_stat_aux[jk] = 0

    Z_bins_XG_CV_stat_median = np.median(Z_bins_XG_CV_stat_aux)   
    Z_bins_XG_CV_eqBperbin_stat.append(Z_bins_XG_CV_stat_median)
    
    Z_bins_XG_CV_stat_stdX = np.std(Z_bins_XG_CV_stat_aux)   
    Z_bins_XG_CV_eqBperbin_stat_std.append(Z_bins_XG_CV_stat_stdX)

    print('# bins: ', B_bins_mean[j_it])
    print('Z median: ', Z_bins_XG_CV_stat_median)
    print('Z std: ', Z_bins_XG_CV_stat_stdX)
    print('')

print(' ')


Z_bins_XG_CV_eqBperbin_stat_plus = [i+j for i, j in zip(Z_bins_XG_CV_eqBperbin_stat, Z_bins_XG_CV_eqBperbin_stat_std)]
Z_bins_XG_CV_eqBperbin_stat_min = [i-j for i, j in zip(Z_bins_XG_CV_eqBperbin_stat, Z_bins_XG_CV_eqBperbin_stat_std)]

### BIN LIKELIHOOD GRID with the 'optimal' number of bins but varying size of bins 

In [ ]:
# SAVED ALL THE ITERATIONS IN:

# metric value
# poiss # asking for a min number of events per bin
# poiss_zeros # replacing the zeros

# Z value
# Z_bins_XG_CV_rd # asking for a min number of events per bin
# Z_bins_XG_CV_rd_zeros # replacing the zeros

In [ ]:
########################################################################
# RANDOM BINNING with CROSS-VAL FOR Nbins and CROSS-VAL for the random #
########################################################################

# divide the test set in 6 sub sets:
# 4 used to compute the grid
# 1 used to compute the metric (chi2 for example)
# 1 used to compute Z

num_SM = int(len(pred_XG_SM)/6)
num_NP = int(len(pred_XG_NP)/6)

numdat = min(num_SM, num_NP)

data_grid_SM_0 = pred_XG_SM[(0*numdat):(1*numdat)]
data_grid_NP_0 = pred_XG_NP[(0*numdat):(1*numdat)]

data_grid_SM_1 = pred_XG_SM[(1*numdat):(2*numdat)]
data_grid_NP_1 = pred_XG_NP[(1*numdat):(2*numdat)]
                            
data_grid_SM_2 = pred_XG_SM[(2*numdat):(3*numdat)]
data_grid_NP_2 = pred_XG_NP[(2*numdat):(3*numdat)]
                          
data_grid_SM_3 = pred_XG_SM[(3*numdat):(4*numdat)]
data_grid_NP_3 = pred_XG_NP[(3*numdat):(4*numdat)]
                                                        
data_grid_SM_4 = pred_XG_SM[(4*numdat):(5*numdat)]
data_grid_NP_4 = pred_XG_NP[(4*numdat):(5*numdat)]

data_grid_SM_5 = pred_XG_SM[(5*numdat):(6*numdat)]
data_grid_NP_5 = pred_XG_NP[(5*numdat):(6*numdat)]






# chi2 = []
# chi2_N = []
# MSE = []
poiss = []
Z_bins_XG_CV_rd = []

# chi2_zeros = []
# chi2_N_zeros = []
# MSE_zeros = []
poiss_zeros = []
Z_bins_XG_CV_rd_zeros = []


# Les't find the number of possible ensembles
N_ensembles_back = len(data_grid_SM_0) / B_expected
N_ensembles_sig = len(data_grid_NP_0) / S_expected


############
# THE GRID #
############

# here j_it iterates over the number of bins
# in this case we are using the 3 'optimal' bin numbers defined above
# you can put a range, i.e.    for j_it in range(10,100):
for j_it in range(len(B_bins_mean)):
    
    print('doing Nbin: ', B_bins_mean[j_it])

    for i_it in range(ntrials):
        
        # bin the parameter space of all background events
        if i_it == 0:
            # for i_it=0 we are going to do equal size bins
            bin_edges = np.linspace( min(pred_XG_SM), max(pred_XG_SM), (B_bins_mean[j_it]+1) )
            
        else:
            # for i_it=/=0 random bin width
            bin_edges = np.hstack( (min(pred_XG_SM), np.sort( np.random.uniform(min(pred_XG_SM), max(pred_XG_SM), (B_bins_mean[j_it]-1)) ), max(pred_XG_SM) ) )
            
        hist_SM_0, _ = np.histogramdd([data_grid_SM_0], bins = [bin_edges])
        hist_SM_1, _ = np.histogramdd([data_grid_SM_1], bins = [bin_edges])
        hist_SM_2, _ = np.histogramdd([data_grid_SM_2], bins = [bin_edges])
        hist_SM_3, _ = np.histogramdd([data_grid_SM_3], bins = [bin_edges])
        
        # bin 4 sub sets, compute the mean to 'average' subsets
        mean = (hist_SM_0 + hist_SM_1 + hist_SM_2 + hist_SM_3 )/4

        if min(mean)>= MIN_EVS*N_ensembles_back:
            
            ##################
            # COMPUTE METRIC #
            ##################
            hist_SM_4, _ = np.histogramdd([data_grid_SM_4], bins = [bin_edges])
            
#             aux_chi2 = sum( ((hist_SM_4 - mean))**2 / mean )
#             aux_chi2_N = aux_chi2 / j_it
#             aux_MSE = sum( ((hist_SM_4 - mean))**2 ) / j_it
            aux_poiss = sum( mean - (hist_SM_4*np.log(mean)) + (hist_SM_4*np.log(hist_SM_4)) - hist_SM_4 )

            
            #############
            # COMPUTE Z #
            #############
            # bin the parameter space of all background events
            hist_SM_5, _ = np.histogramdd([data_grid_SM_5], bins = [bin_edges])
            
            # now divide by the number of possible ensembles
            back_prom_5 = hist_SM_5/N_ensembles_back

            # same for signal
            hist_NP_5, _ = np.histogramdd([data_grid_NP_5], bins = [bin_edges])
            
            sig_prom_5 = hist_NP_5/N_ensembles_sig

            # then the signif Z^binned-Asimov:
            Z_bins_XG_aux = ( 2* sum( ( back_prom_5 * np.log( back_prom_5 / (sig_prom_5+back_prom_5) ) ) + sig_prom_5 ) )**0.5
            
      
        else:
#             aux_chi2 = 9999
#             aux_chi2_N = 9999
#             aux_chi2_sin = 9999
            aux_poiss = 9999
            Z_bins_XG_aux = 0
            

#         chi2.append(aux_chi2)
#         chi2_N.append(aux_chi2_N)
#         MSE.append(aux_chi2_sin)
        poiss.append(aux_poiss)
        Z_bins_XG_CV_rd.append(Z_bins_XG_aux)
        
        
        
        # REPLACING the zeros
        mean_noceros = []
        for i in range(len(mean)):
            if mean[i]!=0:
                mean_noceros.append(mean[i])
        
        min_mean = min(mean_noceros)
        
        # replace the zeros
        for i in range(len(mean)):
            if mean[i]==0:
                mean[i] = min_mean
                
        
        ##################
        # COMPUTE METRIC #
        ##################
        hist_SM_4, _ = np.histogramdd([data_grid_SM_4], bins = [bin_edges])
        
        hist_back_noceros = []
        for i in range(len(hist_SM_4)):
            if hist_SM_4[i]!=0:
                hist_back_noceros.append(hist_SM_4[i])

        min_back = min(hist_back_noceros)

        # replace the zeros
        for i in range(len(hist_SM_4)):
            if hist_SM_4[i]==0:
                hist_SM_4[i] = min_back

#         aux_chi2 = sum( ((hist_SM_4 - mean))**2 / mean )
#         aux_chi2_N = aux_chi2 / j_it
#         aux_MSE = sum( ((hist_SM_4 - mean))**2 ) / j_it
        aux_poiss = sum( mean - (hist_SM_4*np.log(mean)) + (hist_SM_4*np.log(hist_SM_4)) - hist_SM_4 )


        #############
        # COMPUTE Z #
        #############
        # bin the parameter space of all background events
        hist_SM_5, _ = np.histogramdd([data_grid_SM_5], bins = [bin_edges])
        
        hist_back_noceros = []
        for i in range(len(hist_SM_5)):
            if hist_SM_5[i]!=0:
                hist_back_noceros.append(hist_SM_5[i])

        min_back = min(hist_back_noceros)

        # replace the zeros
        for i in range(len(hist_SM_5)):
            if hist_SM_5[i]==0:
                hist_SM_5[i] = min_back

        # now divide by the number of possible ensembles
        back_prom_5 = hist_SM_5/N_ensembles_back

        # same for signal
        hist_NP_5, _ = np.histogramdd([data_grid_NP_5], bins = [bin_edges])

        sig_prom_5 = hist_NP_5/N_ensembles_sig

        # then the signif Z^binned-Asimov:
        Z_bins_XG_aux = ( 2* sum( ( back_prom_5 * np.log( back_prom_5 / (sig_prom_5+back_prom_5) ) ) + sig_prom_5 ) )**0.5

        
        
#         chi2_zeros.append(aux_chi2)
#         chi2_N_zeros.append(aux_chi2_N)
#         MSE_zeros.append(aux_chi2_sin)
        poiss_zeros.append(aux_poiss)
        Z_bins_XG_CV_rd_zeros.append(Z_bins_XG_aux)
        
        
        

#### SELECT THE BINNING with MINIMUM METRIC VALUE

In [ ]:
# SAVED IN:

# metric value
# poiss_per_bin # asking for a min number of events per bin
# poiss_per_bin_zeros # replacing the zeros

# metric value (but bins equal size)
# poiss_per_bin_eqsize # asking for a min number of events per bin
# poiss_per_bin_eqsize_zeros # replacing the zeros

# Z value
# Z_poiss_per_bin # asking for a min number of events per bin
# Z_poiss_per_bin_zeros # replacing the zeros

# Z value (but bins equal size) is done below

In [ ]:
################
# METRIC VALUE #
################

# chi2_per_bin = []
# chi2_N_per_bin = []
# MSE_per_bin = []
poiss_per_bin = []

# chi2_per_bin_eqsize = []
# chi2_N_per_bin_eqsize = []
# MSE_per_bin_eqsize = []
poiss_per_bin_eqsize = []

for i in range(len(B_bins_mean)):
#     chi2_per_bin.append( np.min(chi2[ntrials*i:ntrials*(i+1)]) )
#     chi2_N_per_bin.append( np.min(chi2_N[ntrials*i:ntrials*(i+1)]) )
#     MSE_per_bin.append( np.min(MSE[ntrials*i:ntrials*(i+1)]) )
    poiss_per_bin.append( np.min(poiss[ntrials*i:ntrials*(i+1)]) )
    
#     chi2_per_bin_eqsize.append( chi2[ntrials*i] )
#     chi2_N_per_bin_eqsize.append( chi2_N[ntrials*i] )
#     MSE_per_bin_eqsize.append( MSE[ntrials*i] )
    poiss_per_bin_eqsize.append( poiss[ntrials*i] )
    
    
    
###########
# Z VALUE #
###########

# Z_chi2_per_bin = []
# Z_chi2_N_per_bin = []
# Z_MSE_per_bin = []
Z_poiss_per_bin = []

for i in range(len(B_bins_mean)):
#     Z_chi2_per_bin.append( Z_bins_XG_CV_rd[np.argmin(chi2[ntrials*i:ntrials*(i+1)]) + ntrials*i] )
#     Z_chi2_N_per_bin.append( Z_bins_XG_CV_rd[np.argmin(chi2_N[ntrials*i:ntrials*(i+1)]) + ntrials*i] )
#     Z_MSE_per_bin.append( Z_bins_XG_CV_rd[np.argmin(MSE[ntrials*i:ntrials*(i+1)]) + ntrials*i] )
    Z_poiss_per_bin.append( Z_bins_XG_CV_rd[np.argmin(poiss[ntrials*i:ntrials*(i+1)]) + ntrials*i] )  
    
    
    
# SAME REPLACING THE ZEROS

################
# METRIC VALUE #
################

# chi2_per_bin_zeros = []
# chi2_N_per_bin_zeros = []
# MSE_per_bin_zeros = []
poiss_per_bin_zeros = []

# chi2_per_bin_eqsize_zeros = []
# chi2_N_per_bin_eqsize_zeros = []
# MSE_per_bin_eqsize_zeros = []
poiss_per_bin_eqsize_zeros = []

for i in range(len(B_bins_mean)):
#     chi2_per_bin_zeros.append( np.min(chi2_zeros[ntrials*i:ntrials*(i+1)]) )
#     chi2_N_per_bin_zeros.append( np.min(chi2_N_zeros[ntrials*i:ntrials*(i+1)]) )
#     MSE_per_bin_zeros.append( np.min(MSE_zeros[ntrials*i:ntrials*(i+1)]) )
    poiss_per_bin_zeros.append( np.min(poiss_zeros[ntrials*i:ntrials*(i+1)]) )
    
#     chi2_per_bin_eqsize_zeros.append( chi2_zeros[ntrials*i] )
#     chi2_N_per_bin_eqsize_zeros.append( chi2_N_zeros[ntrials*i] )
#     MSE_per_bin_eqsize_zeros.append( MSE_zeros[ntrials*i] )
    poiss_per_bin_eqsize_zeros.append( poiss_zeros[ntrials*i] )
    
    
###########
# Z VALUE #
###########

# Z_chi2_per_bin_zeros = []
# Z_chi2_N_per_bin_zeros = []
# Z_MSE_per_bin_zeros = []
Z_poiss_per_bin_zeros = []

for i in range(len(B_bins_mean)):
#     Z_chi2_per_bin_zeros.append( Z_bins_XG_CV_rd_zeros[np.argmin(chi2_zeros[ntrials*i:ntrials*(i+1)]) + ntrials*i] )
#     Z_chi2_N_per_bin_zeros.append( Z_bins_XG_CV_rd_zeros[np.argmin(chi2_N_zeros[ntrials*i:ntrials*(i+1)]) + ntrials*i] )
#     Z_MSE_per_bin_zeros.append( Z_bins_XG_CV_rd_zeros[np.argmin(MSE_zeros[ntrials*i:ntrials*(i+1)]) + ntrials*i] )
    Z_poiss_per_bin_zeros.append( Z_bins_XG_CV_rd_zeros[np.argmin(poiss_zeros[ntrials*i:ntrials*(i+1)]) + ntrials*i] )   
    

####  SELECT THE BINNING with MAXIMUM Z

In [ ]:
# SAVED IN:

# Z value
# maxZ_per_bin # asking for a min number of events per bin
# maxZ_per_bin_zeros # replacing the zeros

# Z value (but bins equal size) (SHOULD BE THE SAME AS IN THE EQ SIZE SECTION)
# maxZ_per_bin_eqsize # asking for a min number of events per bin
# maxZ_per_bin_eqsize_zeros # replacing the zeros

In [ ]:
###########
# Z VALUE #
###########

maxZ_per_bin = []
maxZ_per_bin_eqsize = []

for i in range(len(B_bins_mean)):
    maxZ_per_bin.append( np.max(Z_bins_XG_CV_rd[ntrials*i:ntrials*(i+1)]) )
    maxZ_per_bin_eqsize.append( Z_bins_XG_CV_rd[ntrials*i] )
    

# SAME REPLACING THE ZEROS


maxZ_per_bin_zeros = []
maxZ_per_bin_eqsize_zeros = []

for i in range(len(B_bins_mean)):
    maxZ_per_bin_zeros.append( np.max(Z_bins_XG_CV_rd_zeros[ntrials*i:ntrials*(i+1)]) )
    maxZ_per_bin_eqsize_zeros.append( Z_bins_XG_CV_rd_zeros[ntrials*i] )

### PLOT

In [ ]:
###################
# EQUAL SIZE BINS #
###################

# Z_bins_XG_CV # asking for a min number of events per bin (NO STAT ERROR)
# Z_bins_XG_CV_zeros # replacing the zeros (NO STAT ERROR)


# Z_bins_XG_CV_stat # asking for a min number of events per bin
# Z_bins_XG_CV_stat_plus # central value +1sigma
# Z_bins_XG_CV_stat_min # central value -1sigma

# Z_bins_XG_CV_stat_zeros # replacing the zeros
# Z_bins_XG_CV_stat_zeros_plus # central value +1sigma
# Z_bins_XG_CV_stat_zeros_min # central value -1sigma


#############################################
# EQUAL NUMBER OF BACKGROUND EVENTS per BIN #
#############################################

# Z_bins_XG_CV_eqBperbin # asking for a min number of events per bin (NO STAT ERROR)
# no replacing zeros because all bins have the same number of events


# Z_bins_XG_CV_eqBperbin_stat # asking for a min number of events per bin
# Z_bins_XG_CV_eqBperbin_stat_plus # central value +1sigma
# Z_bins_XG_CV_eqBperbin_stat_min # central value -1sigma


#######################
# MINIMIZING A METRIC #
#######################

# Z_poiss_per_bin # asking for a min number of events per bin
# Z_poiss_per_bin_zeros # replacing the zeros


################
# MAXIMIZING Z #
################

# maxZ_per_bin # asking for a min number of events per bin
# maxZ_per_bin_zeros # replacing the zeros

# equal size (SHOULD BE THE SAME AS IN THE EQ SIZE SECTION)
# maxZ_per_bin_eqsize # asking for a min number of events per bin
# maxZ_per_bin_eqsize_zeros # replacing the zeros

In [ ]:
# CHECK

print(' check that the grid works, this should be identity ')

plt.plot(maxZ_per_bin_eqsize, Z_bins_XG_CV, '.', color = 'red', label='Equal size bins')
plt.plot(maxZ_per_bin_eqsize_zeros, Z_bins_XG_CV_zeros, '.', color = 'magenta', label='Equal size bins (zeros)')

#plt.title('')
plt.xlabel('Z', fontsize = 12)
plt.ylabel('Z', fontsize = 12)
#plt.ylim([5.6,6.8])
plt.legend(fontsize = 12)
#plt.savefig(rootir+'random-10k.pdf')
plt.show()

In [ ]:
#########
# PLOTS #
#########

print('fd', B_bins_mean[0])
print('doane', B_bins_mean[1])
print('sturges', B_bins_mean[2])


plt.plot(B_bins_mean, Z_bins_XG_CV, '.', color = 'red', label='Equal size bins')
plt.plot(B_bins_mean, Z_bins_XG_CV_stat, 'x', color = 'red', label='Equal size bins stat')
plt.plot(B_bins_mean, Z_bins_XG_CV_stat_plus, 'x', color = 'red')
plt.plot(B_bins_mean, Z_bins_XG_CV_stat_min, 'x', color = 'red')

plt.plot(B_bins_mean, Z_bins_XG_CV_zeros, '.', color = 'magenta', label='Equal size bins (zeros)')
plt.plot(B_bins_mean, Z_bins_XG_CV_stat_zeros, 'x', color = 'magenta', label='Equal size bins stat (zeros)')
plt.plot(B_bins_mean, Z_bins_XG_CV_stat_zeros_plus, 'x', color = 'magenta')
plt.plot(B_bins_mean, Z_bins_XG_CV_stat_zeros_min, 'x', color = 'magenta')

plt.plot(B_bins_mean, Z_bins_XG_CV_eqBperbin, '.', color = 'blue', label='Equal background events per bin')
plt.plot(B_bins_mean, Z_bins_XG_CV_eqBperbin_stat, 'x', color = 'blue', label='Equal background events per bin stat')
plt.plot(B_bins_mean, Z_bins_XG_CV_eqBperbin_stat_plus, 'x', color = 'blue')
plt.plot(B_bins_mean, Z_bins_XG_CV_eqBperbin_stat_min, 'x', color = 'blue')

plt.plot(B_bins_mean, Z_poiss_per_bin, '.', color = 'green', label='Random bins (min metric)')
plt.plot(B_bins_mean, Z_poiss_per_bin_zeros, '.', color = 'purple', label='Random bins (min metric) (zeros)')
plt.plot(B_bins_mean, maxZ_per_bin, '.', color = 'darkorange', label='Random bins (max Z)')
plt.plot(B_bins_mean, maxZ_per_bin_zeros, '.', color = 'gray', label='Random bins (max Z) (zeros)')

#plt.title('')
plt.xlabel('$N_{bin}$', fontsize = 12)
plt.ylabel('Z', fontsize = 12)
#plt.ylim([5.6,6.8])
plt.legend(fontsize = 12)
#plt.savefig(rootir+'random-10k.pdf')
plt.show()

## MLL+KDE

### KDE

In [ ]:
# USE KDE TO ESTIMATE THE CLASSIFIER OUTPUT PDFs



# FIND THE BANDWIDTH  # CHECK IF GOOD RESULTS WITH THIS PARAMETERS
bandwidth = np.logspace(-4.0, 0.05, 50)
num_evs_toKDE = 20000


kde = KernelDensity(kernel='epanechnikov')
grid = GridSearchCV(kde, {'bandwidth': bandwidth})
grid.fit(np.c_[pred_XG_SM[:num_evs_toKDE]])
print('Background: ', grid.best_estimator_)

SM_bandwidth = grid.best_estimator_.bandwidth



kde = KernelDensity(kernel='epanechnikov')
grid = GridSearchCV(kde, {'bandwidth': bandwidth})
grid.fit(np.c_[pred_XG_NP[:num_evs_toKDE]])
print('Signal: ', grid.best_estimator_)

NP_bandwidth = grid.best_estimator_.bandwidth



# with each calculated bandwidth estimate the pdf with KDE to the classifier output (for background and signal)
# notice: epanechnikov kernel
kde_bkg = KernelDensity(kernel="epanechnikov", bandwidth=SM_bandwidth).fit(np.c_[pred_XG_SM, np.zeros(len(pred_XG_SM)) ])
kde_sig = KernelDensity(kernel="epanechnikov", bandwidth=NP_bandwidth).fit(np.c_[pred_XG_NP, np.ones(len(pred_XG_NP)) ])



# TO NORMALIZE TO 1, COMPUTE THE AREA

# range
min_val = np.min([np.min(pred_XG_SM),np.min(pred_XG_NP)])
max_val = np.max([np.max(pred_XG_SM),np.max(pred_XG_NP)])

s_vals = np.linspace(min_val,max_val,1000)


# evaluate the densities for each value of s (~bins)
dens_bkg = np.exp(kde_bkg.score_samples(np.c_[s_vals, np.zeros(len(s_vals)) ]) )
dens_sig = np.exp(kde_sig.score_samples(np.c_[s_vals, np.ones(len(s_vals)) ]) )


# Area
factor_aux_SM = sum(dens_bkg*(s_vals[1]-s_vals[0]))
factor_aux_NP = sum(dens_sig*(s_vals[1]-s_vals[0]))

# normalize
dens_bkg = dens_bkg / factor_aux_SM
dens_sig = dens_sig / factor_aux_NP



# plot to check the estimation 

plt.figure(figsize=(7,5))

plt.hist(pred_XG_SM, 25, range=[0,1], density=True, color='blue',alpha=0.5, linewidth=2, label=r'Binned $\tilde{p}_{b}(o(x))$');
plt.hist(pred_XG_NP, 25, range=[0,1], density=True, color='red',alpha=0.5, linewidth=2, label=r'Binned $\tilde{p}_{s}(o(x))$');

plt.plot(s_vals,dens_bkg,color='blue',label=r'KDE $\tilde{p}_{b}(o(x))$',linestyle='dashed');
plt.plot(s_vals,dens_sig,color='red',label=r'KDE $\tilde{p}_{s}(o(x))$',linestyle='dashed');

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.yscale('log')
plt.xlabel("Classication score (XGBoost)",fontsize=14)
plt.ylabel("Fraction of events/bin size",fontsize=14)
plt.grid()
plt.legend(loc='upper center',fontsize=13)
plt.show()

### MLL+KDE

In [ ]:
# SAVED IN:

# store_Z_MLL_KDE   # Z for MLL+KDE
# store_Z_MLL_KDE_plus # central value +1sigma statistic
# store_Z_MLL_KDE_min # central value -1sigma statistic

# same but fixing mu=0
# store_Z_MLL_KDE_mu0
# store_Z_MLL_KDE_mu0_plus
# store_Z_MLL_KDE_mu0_min

In [ ]:
# FOR EXCLUSION:
# we need to evaluate the KDE densities with the classifier output of BACKGROUND EVENTS

# FOR DISCOVERY:
# we need to evaluate the KDE densities with the classifier output of BACKGROUND and SIGNAL EVENTS

In [ ]:
#####################################################################
# EVALUATE EVERY POINT IN THE KDE (its faster to de it all at once) #
#####################################################################

# evaluate every BACKGROUND point in the BACKGROUND PDF (obtained with KDE)
KDE_SM_pred_SM = np.exp(kde_bkg.score_samples(np.c_[pred_XG_SM, np.zeros(len(pred_XG_SM)) ]) )

# evaluate every BACKGROUND point in the SIGNAL (yes signal) PDF (obtained with KDE)
KDE_NP_pred_SM = np.exp(kde_sig.score_samples(np.c_[pred_XG_SM, np.ones(len(pred_XG_SM)) ]) )


# Normalize

KDE_SM_pred_SM = KDE_SM_pred_SM / factor_aux_SM
KDE_NP_pred_SM = KDE_NP_pred_SM / factor_aux_NP

In [ ]:
store_muhat_mean_MLL_KDE = []

store_Z_MLL_KDE = []
store_Z_MLL_KDE_mu0 = []

store_Z_MLL_KDE_std = []
store_Z_MLL_KDE_std_mu0 = []


indices = [i for i in range(len(KDE_NP_pred_SM))]


# this loops computes everything for different cases
# for example, for different multiple of S_expected = [100,200,300]
# replace    for iii in range(0,1):     --->    for iii in range(len(S_expected)):
# and replace everywhere    S_expected  --->    S_expected[iii]
for iii in range(0,1):
    
    print('B_expected: ', B_expected)
    print('S_expected: ', S_expected)
    print('n_ensembles (initial): ', n_ensembles)


    # to construct ensembles B and S events are taken from Poisson distributions
    mu = S_expected + B_expected


    # Letś find the number of events per ensemble such that we get at least one ensemble populated if events are taken from a Poisson distribution

    # around the mean its populated so let's try (proposed range to be checked)
    list_events_per_ensembles = [i for i in range(int(mu*0.9),int(mu*1.1))]
    to_check = len(list_events_per_ensembles)

    # I want at least one ensemble populated
    list_nums_ensembles = [ int( poisson.pmf(list_events_per_ensembles[i],mu)*n_ensembles ) for i in range(len(list_events_per_ensembles)) ]



    # Remove from the list the elements without at least 1 ensemble possible
    for i in range(len(list_events_per_ensembles)):
        if list_nums_ensembles[i] > 1:
            list_events_per_ensembles = list_events_per_ensembles[i:]
            list_nums_ensembles = list_nums_ensembles[i:]
            break


    for i in range(len(list_events_per_ensembles)):
        if list_nums_ensembles[i] < 1:
            list_events_per_ensembles = list_events_per_ensembles[:i]
            list_nums_ensembles = list_nums_ensembles[:i]
            break

    print('\n If ', to_check, ' = ', len(list_events_per_ensembles), '   then the proposed range has to be extended')

    print('n_ensembles (actual): ', sum(list_nums_ensembles))



    # lists of S and B events per ensemble, w.r.t the total of number of events per ensemble found above:

    p_berno = S_expected/(S_expected+B_expected)

    list_S_per_ensembles = []
    list_B_per_ensembles = []

    for jj in range(len(list_events_per_ensembles)):
        list_S_per_ensembles.append( int(p_berno * list_events_per_ensembles[jj]) )
        list_B_per_ensembles.append( list_events_per_ensembles[jj] - int(p_berno * list_events_per_ensembles[jj]) )

    ######
    # NOW I HAVE 4 LISTS:
    # list_events_per_ensembles     list with the number of events per ensemble (its a range)
    # list_nums_ensembles           list with the number of ensembles, w.r.t the 1st list
    # list_S_per_ensembles          list with the number of signal events in each ensembles, w.r.t the 1st list
    # list_B_per_ensembles          list with the number of background events in each ensembles, w.r.t the 1st list
    ######



    print('\n This may take long... \n')
    
    
    #############################
    # NOW LETS APPLY THE METHOD #
    #############################

    muhat_selected_KDE_list = []
    q_muhat_KDE = []
    q_muhat_KDE_mu0 = []
    
    for bb in range(len(list_nums_ensembles)):

        for kk in range(list_nums_ensembles[bb]):
            
            # KDE
            ran_ind = np.random.choice(indices, list_B_per_ensembles[bb])

            KDE_SM_pred_SM_shuf = []
            KDE_NP_pred_SM_shuf = []

            for i in ran_ind:
                KDE_SM_pred_SM_shuf.append(KDE_SM_pred_SM[i])
                KDE_NP_pred_SM_shuf.append(KDE_NP_pred_SM[i])

            KDE_SM_pred_SM_shuf  = np.array(KDE_SM_pred_SM_shuf)
            KDE_NP_pred_SM_shuf  = np.array(KDE_NP_pred_SM_shuf)





            # p_b(o(x_ensemble)) =    concatenate: p_b(o(B_ensemble)) and p_b(o(S_ensemble))  NOTICE THE o(x)
            prob_x_given_B = np.ndarray.tolist( KDE_SM_pred_SM_shuf ) #+ np.ndarray.tolist( KDE_SM_pred_NP_shuf )

            # p_s(o(x_ensemble)) =    concatenate: p_s(o(B_ensemble)) and p_s(o(S_ensemble))  NOTICE THE o(x)
            prob_x_given_S = np.ndarray.tolist( KDE_NP_pred_SM_shuf ) #+ np.ndarray.tolist( KDE_NP_pred_NP_shuf )



            

            # NOW WE HAVE p_{s,b}(x_ensemble) for this particular ensemble
            # WE NEED TO ESTIMATE mu_hat for this particular ensemble
            # we are going to obtain a mu_hat with a grid of values for this particular ensemble

            B_prob_x_given_B = [x * B_expected for x in prob_x_given_B]
            
            sum_muhat_zero = sum ( [(x*1.) / ( (x * 0. * S_expected) + y ) for x, y in zip(prob_x_given_S, B_prob_x_given_B)] )
            sum_muhat_one = sum ( [(x*1.) / ( (x * 1. * S_expected) + y ) for x, y in zip(prob_x_given_S, B_prob_x_given_B)] )

            # grid, mu_hat is around 1
            muhat_test = np.arange(0., 1., 0.05).tolist()

            muhat_selected_KDE = 0.0

            if sum_muhat_zero < sum_muhat_one and sum_muhat_zero < 1:

                for vv in range(len(muhat_test)):

                    mu_hat_condition_equal_1 = sum ( [(x*1.) / ( (x * muhat_test[vv] * S_expected) + y ) for x, y in zip(prob_x_given_S, B_prob_x_given_B)] )

                    if mu_hat_condition_equal_1 > 1:
                        muhat_selected_KDE = muhat_test[vv]
                        break

            elif sum_muhat_zero > sum_muhat_one and sum_muhat_zero > 1:

                for vv in range(len(muhat_test)):

                    mu_hat_condition_equal_1 = sum ( [(x*1.) / ( (x * muhat_test[vv] * S_expected) + y ) for x, y in zip(prob_x_given_S, B_prob_x_given_B)] )

                    if mu_hat_condition_equal_1 < 1:
                        muhat_selected_KDE = muhat_test[vv]
                        break


            muhat_selected_KDE_list.append(muhat_selected_KDE)



            # NOW THAT WE HAVE mu_hat FOR THIS ENSEMBLE, CALCULATE THE TEST STATISTIC FOR THIS ENSEMBLE
            # and append it (we need the median over lots of ensembles)
            #q_muhat.append( 2 * ( (-1.*muhat_selected * S_expected) + sum( [np.log( 1 + ( (muhat_selected*S_expected/B_expected) * (x / y) ) ) for x, y in zip(prob_x_given_S, prob_x_given_B)] ) ) )
            #q_muhat_mu0.append( 2 * ( (-1.*1. * S_expected) + sum( [np.log( 1 + ( (1.*S_expected/B_expected) * (x / y) ) ) for x, y in zip(prob_x_given_S, prob_x_given_B)] ) ) )
            # EXCLUSION:
            q_muhat_KDE.append( 2 * ( ( (1.-muhat_selected_KDE) * S_expected ) - sum( [np.log( ( (B_expected*y) + (S_expected*x) ) / ( (B_expected*y) + (muhat_selected_KDE*S_expected*x) ) ) for x, y in zip(prob_x_given_S, prob_x_given_B)] ) ) )
            q_muhat_KDE_mu0.append( 2 * ( ( (1.-0.) * S_expected ) - sum( [np.log( ( (B_expected*y) + (S_expected*x) ) / ( (B_expected*y) + (0.*S_expected*x) ) ) for x, y in zip(prob_x_given_S, prob_x_given_B)] ) ) )


    # Histogram of q_muhats

    weights = np.ones_like(q_muhat_KDE)/float(len(q_muhat_KDE))
    nMIX, binsMIX, patchesMIX = plt.hist(q_muhat_KDE, 25, weights=weights, histtype='step', color='blue', linewidth=2)
    #plt.xlim(0,1)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlabel("$q_0(\hat{\mu})$",fontsize=16)
    plt.ylabel("Fraction of ensembles",fontsize=16)
    plt.title(r"$\bar{B}$=%0.2i, $\bar{S}=$%0.2i" % (B_expected,S_expected),fontsize=14)
    plt.grid()
    #plt.legend(fontsize=14)
    plt.show()


    # Finally calculate muhat_mean and Z_gaussian
    
    # Remove nan if any
    q_muhat_KDE_mu0 = [x for x in q_muhat_KDE_mu0 if x == x]
    for jk in range(len(q_muhat_KDE_mu0)):
        if q_muhat_KDE_mu0[jk] < 0:
            q_muhat_KDE_mu0[jk] = 0

    q_muhat_KDE_median_mu0 = np.median(q_muhat_KDE_mu0)
    Z_KDE_mu0 = abs(q_muhat_KDE_median_mu0)**0.5
    store_Z_MLL_KDE_mu0.append(Z_KDE_mu0)
    
    q_muhat_KDE_std_mu0 = np.std(q_muhat_KDE_mu0)
    Z_KDE_std_mu0 = q_muhat_KDE_std_mu0/(2.*Z_KDE_mu0)
    store_Z_MLL_KDE_std_mu0.append(Z_KDE_std_mu0)
    

    # Finally calculate muhat_mean and Z_gaussian
    muhat_mean_MLL_KDE = np.mean(muhat_selected_KDE_list)
    store_muhat_mean_MLL_KDE.append(muhat_mean_MLL_KDE)

    # Remove nan if any
    q_muhat_KDE = [x for x in q_muhat_KDE if x == x]
    for jk in range(len(q_muhat_KDE)):
        if q_muhat_KDE[jk] < 0:
            q_muhat_KDE[jk] = 0

    q_muhat_KDE_median = np.median(q_muhat_KDE)
    Z_KDE = abs(q_muhat_KDE_median)**0.5
    store_Z_MLL_KDE.append(Z_KDE)
    
    q_muhat_KDE_std = np.std(q_muhat_KDE)
    Z_KDE_std = q_muhat_KDE_std/(2.*Z_KDE)
    store_Z_MLL_KDE_std.append(Z_KDE_std)

    print('muhat mean: ', muhat_mean_MLL_KDE)
    print('median q_muhat_KDE: ', q_muhat_KDE_median)
    print('Z_KDE: ', Z_KDE)
    print('Z_KDE mu=0: ', Z_KDE_mu0)
    print('std Z_KDE: ', Z_KDE_std)
    print('std Z_KDE mu=0: ', Z_KDE_std_mu0)

    print('\n -------------------------------- \n')
    
    
store_Z_MLL_KDE_plus = [i+j for i, j in zip(store_Z_MLL_KDE, store_Z_MLL_KDE_std)]
store_Z_MLL_KDE_min = [i-j for i, j in zip(store_Z_MLL_KDE, store_Z_MLL_KDE_std)]

store_Z_MLL_KDE_mu0_plus = [i+j for i, j in zip(store_Z_MLL_KDE_mu0, store_Z_MLL_KDE_std_mu0)]
store_Z_MLL_KDE_mu0_min = [i-j for i, j in zip(store_Z_MLL_KDE_mu0, store_Z_MLL_KDE_std_mu0)]

### PLOT

In [ ]:
# -------------- #
# BIN LIKELIHOOD #
# -------------- #

###################
# EQUAL SIZE BINS #
###################

# Z_bins_XG_CV # asking for a min number of events per bin (NO STAT ERROR)
# Z_bins_XG_CV_zeros # replacing the zeros (NO STAT ERROR)


# Z_bins_XG_CV_stat # asking for a min number of events per bin
# Z_bins_XG_CV_stat_plus # central value +1sigma
# Z_bins_XG_CV_stat_min # central value -1sigma

# Z_bins_XG_CV_stat_zeros # replacing the zeros
# Z_bins_XG_CV_stat_zeros_plus # central value +1sigma
# Z_bins_XG_CV_stat_zeros_min # central value -1sigma


#############################################
# EQUAL NUMBER OF BACKGROUND EVENTS per BIN #
#############################################

# Z_bins_XG_CV_eqBperbin # asking for a min number of events per bin (NO STAT ERROR)
# no replacing zeros because all bins have the same number of events


# Z_bins_XG_CV_eqBperbin_stat # asking for a min number of events per bin
# Z_bins_XG_CV_eqBperbin_stat_plus # central value +1sigma
# Z_bins_XG_CV_eqBperbin_stat_min # central value -1sigma


#######################
# MINIMIZING A METRIC #
#######################

# Z_poiss_per_bin # asking for a min number of events per bin
# Z_poiss_per_bin_zeros # replacing the zeros


################
# MAXIMIZING Z #
################

# maxZ_per_bin # asking for a min number of events per bin
# maxZ_per_bin_zeros # replacing the zeros

# equal size (SHOULD BE THE SAME AS IN THE EQ SIZE SECTION)
# maxZ_per_bin_eqsize # asking for a min number of events per bin
# maxZ_per_bin_eqsize_zeros # replacing the zeros



# ------- #
# MLL+KDE #
# ------- #

# store_Z_MLL_KDE   # Z for MLL+KDE
# store_Z_MLL_KDE_plus # central value +1sigma statistic
# store_Z_MLL_KDE_min # central value -1sigma statistic

# same but fixing mu=0
# store_Z_MLL_KDE_mu0
# store_Z_MLL_KDE_mu0_plus
# store_Z_MLL_KDE_mu0_min

In [ ]:
#########
# PLOTS #
#########

print('fd', B_bins_mean[0])
print('doane', B_bins_mean[1])
print('sturges', B_bins_mean[2])


# PLOT KDE IN BIN=0 (ACTUALLY UNBIN METHOD)
no_bin_MLLKDE = [0]




plt.plot(B_bins_mean, Z_bins_XG_CV, '.', color = 'red', label='Equal size bins')
plt.plot(B_bins_mean, Z_bins_XG_CV_stat, 'x', color = 'red', label='Equal size bins stat')
plt.plot(B_bins_mean, Z_bins_XG_CV_stat_plus, 'x', color = 'red')
plt.plot(B_bins_mean, Z_bins_XG_CV_stat_min, 'x', color = 'red')

plt.plot(B_bins_mean, Z_bins_XG_CV_zeros, '.', color = 'magenta', label='Equal size bins (zeros)')
plt.plot(B_bins_mean, Z_bins_XG_CV_stat_zeros, 'x', color = 'magenta', label='Equal size bins stat (zeros)')
plt.plot(B_bins_mean, Z_bins_XG_CV_stat_zeros_plus, 'x', color = 'magenta')
plt.plot(B_bins_mean, Z_bins_XG_CV_stat_zeros_min, 'x', color = 'magenta')

plt.plot(B_bins_mean, Z_bins_XG_CV_eqBperbin, '.', color = 'blue', label='Equal background events per bin')
plt.plot(B_bins_mean, Z_bins_XG_CV_eqBperbin_stat, 'x', color = 'blue', label='Equal background events per bin stat')
plt.plot(B_bins_mean, Z_bins_XG_CV_eqBperbin_stat_plus, 'x', color = 'blue')
plt.plot(B_bins_mean, Z_bins_XG_CV_eqBperbin_stat_min, 'x', color = 'blue')

plt.plot(B_bins_mean, Z_poiss_per_bin, '.', color = 'green', label='Random bins (min metric)')
plt.plot(B_bins_mean, Z_poiss_per_bin_zeros, '.', color = 'purple', label='Random bins (min metric) (zeros)')
plt.plot(B_bins_mean, maxZ_per_bin, '.', color = 'darkorange', label='Random bins (max Z)')
plt.plot(B_bins_mean, maxZ_per_bin_zeros, '.', color = 'gray', label='Random bins (max Z) (zeros)')

plt.plot(no_bin_MLLKDE, store_Z_MLL_KDE, 'x', color='pink', lw=lw, label='MLL+KDE stat')
plt.plot(no_bin_MLLKDE, store_Z_MLL_KDE_plus, 'x', color='pink', lw=lw)
plt.plot(no_bin_MLLKDE, store_Z_MLL_KDE_min, 'x', color='pink', lw=lw)

#plt.title('')
plt.xlabel('$N_{bin}$', fontsize = 12)
plt.ylabel('Z', fontsize = 12)
#plt.ylim([5.6,6.8])
plt.legend(fontsize = 12)
#plt.savefig(rootir+'random-10k.pdf')
plt.show()